In [17]:
import nltk
from nltk.corpus import movie_reviews
import random
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
import string
from sklearn.model_selection import train_test_split

In [3]:
movie_reviews.categories()
lemmatizer=WordNetLemmatizer()

In [4]:
stops=stopwords.words('english')
punc=list(string.punctuation)
stops=stops+punc

In [5]:
len(movie_reviews.fileids())
#1000 -ve
#1000 +ve       

2000

In [6]:
#it returns file id with positive reviews
pos_rev=movie_reviews.fileids('pos')

In [7]:
#we need to pass the file id here to get words
movie_reviews.words(movie_reviews.fileids())

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

In [8]:
documents=[]
for category in movie_reviews.categories():
    for fid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fid),category))
documents[1]        

(['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg')

***Shuffling to mix pos/neg so that it becomes easy for train_test_split***

In [9]:
random.shuffle(documents)

In [10]:
def getsimplepos (tag) :
    if tag.startswith( 'J'):
        return wordnet.ADJ
    elif tag.startswith( 'V'):
        return wordnet. VERB
    elif tag.startswith( 'N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet .NOUN

**pos_tag requires an array**

In [11]:
w="better"
pos_tag([w])

[('better', 'RBR')]

**Cleaning the dataset**

In [12]:
def clean_review(words):
    output_words=[]
    for w in words:
        if(w.lower() not in stops):
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=getsimplepos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words        
            

In [13]:
documents=[(clean_review(document),category) for document,category in documents]

In [14]:
documents[0]

(['near',
  'end',
  'frank',
  'capra',
  'holiday',
  'classic',
  'wonderful',
  'life',
  'george',
  'bailey',
  'jimmy',
  'stewart',
  'feel',
  'devote',
  'life',
  'others',
  'never',
  'fulfil',
  'dream',
  'leave',
  'small',
  'supburb',
  'town',
  'bedford',
  'fall',
  'stand',
  'upon',
  'bridge',
  'stormy',
  'water',
  'look',
  'contemplate',
  'suicide',
  'potential',
  'fulfil',
  '--',
  'yet',
  'do',
  'much',
  'others',
  'young',
  'chap',
  'ambition',
  'leave',
  'hometown',
  'bedford',
  'fall',
  'time',
  'ambition',
  'change',
  'thing',
  'happen',
  'stuck',
  'guardian',
  'angel',
  'show',
  'wonderful',
  'life',
  'whirlwind',
  'tour',
  'town',
  'george',
  'see',
  'life',
  'would',
  'like',
  'without',
  'learns',
  'moral',
  'lesson',
  'live',
  'gift',
  'earthly',
  'problem',
  'solve',
  'climatic',
  'scene',
  'marvel',
  'frank',
  'capra',
  'film',
  'centenial',
  'classic',
  '--',
  'watch',
  'experienced',
  'eve

**Now creating dataset with feature as top words**

***Building the feature sets only on training data***

In [15]:
training=documents[:1500]
testing=documents[1500:]

In [20]:
#we will collect all words in an array and find frequency distribution
all_words=[]
for doc in training:
    all_words+=doc[0]


In [23]:
freq=nltk.FreqDist(all_words)
common=freq.most_common(3000)

In [25]:
features=[words for words,count in common]
#features

In [29]:
def get_feature_dict(words):
    current_features={}
    w_set=set(words)
    for w in features:
        current_features[w]=w in w_set
    return current_features    

In [31]:
training_data=[(get_feature_dict(word),category)for word,category in training]
testing_data=[(get_feature_dict(word),category)for word,category in testing]

In [32]:
training_data[0]

({'film': True,
  'movie': False,
  'one': True,
  'make': False,
  'like': True,
  'character': False,
  'get': False,
  'see': True,
  'time': True,
  'go': True,
  'well': True,
  'scene': True,
  'even': True,
  'good': True,
  'story': False,
  'take': False,
  'would': True,
  'much': True,
  'come': False,
  'also': False,
  'bad': True,
  'life': True,
  'two': False,
  'give': False,
  'end': True,
  'look': True,
  '--': True,
  'way': False,
  'know': False,
  'first': True,
  'seem': True,
  'year': False,
  'work': False,
  'thing': True,
  'plot': False,
  'play': False,
  'say': False,
  'really': False,
  'little': True,
  'people': False,
  'show': True,
  'could': False,
  'star': False,
  'man': False,
  'love': True,
  'director': False,
  'performance': False,
  'try': False,
  'never': True,
  'new': False,
  'best': False,
  'great': False,
  'big': False,
  'many': True,
  'u': False,
  'watch': True,
  'actor': True,
  'want': False,
  'find': False,
  'action'

**Using NaiveBayes classifier in nltk**

In [33]:
from nltk import NaiveBayesClassifier

In [34]:
classifier=NaiveBayesClassifier.train(training_data)

In [35]:
nltk.classify.accuracy(classifier,testing_data)

0.768

In [37]:
classifier.show_most_informative_features()

Most Informative Features
             outstanding = True              pos : neg    =     18.0 : 1.0
               ludicrous = True              neg : pos    =     13.4 : 1.0
               stupidity = True              neg : pos    =     10.3 : 1.0
                  turkey = True              neg : pos    =      9.9 : 1.0
                  seagal = True              neg : pos    =      9.4 : 1.0
                   mulan = True              pos : neg    =      7.2 : 1.0
             magnificent = True              pos : neg    =      7.2 : 1.0
             wonderfully = True              pos : neg    =      6.5 : 1.0
                    diaz = True              pos : neg    =      6.4 : 1.0
                  welles = True              neg : pos    =      6.1 : 1.0
